In [1]:
import torch
import torchvision
import torch.optim as optim
from data.data import get_train_test_loader
from model.network import FastUpdateNet, TeacherNet
import torch.nn.functional as F
import time
import pickle

/home/kyle/pythonenvs/venv38/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
n_epochs = 20
batch_size_train = 512
batch_size_test = 1000
learning_rate = 0.0001
momentum = 0.5
log_interval = 10

random_seed = 1
torch.backends.cudnn.enabled = False
# torch.manual_seed(random_seed)

In [3]:
train_loader, test_loader = get_train_test_loader('mnist', batch_size_train = batch_size_train)

In [4]:
train_losses = []
train_counter = []
test_losses = []
test_accuracies = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]

In [5]:
from threading import Thread

def do(network):
  network.mNet.backwardHidden() 

def train_fastNet(epoch, network):
  network.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    optimizer.zero_grad()
    output = network(data)
    loss = F.nll_loss(output, target)
    try:
      loss.backward()
    except RuntimeError:
      continue
    # print(network.mNet.saver.grad)
    p1 = Thread(target=do, args=[network]) # start two independent threads

    p1.start()
        
    p1.join()
    # split into threads
    correctness = (target == torch.argmax(output))
    optimizer.step()
    if batch_idx % log_interval == 0:
      train_losses.append(loss.item())
      train_counter.append(
        (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))


def train_teacherNet(epoch, network):
  network.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    optimizer.zero_grad()
    output = network(data)
    loss = F.nll_loss(output, target)
    loss.backward()
    # print(network.mNet.saver.grad)
    correctness = (target == torch.argmax(output))
    optimizer.step()
    if batch_idx % log_interval == 0:
      train_losses.append(loss.item())
      train_counter.append(
        (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))

In [6]:
def test(network):
  network.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:
      output = network(data)
      test_loss += F.nll_loss(output, target, size_average=False).item()
      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(target.data.view_as(pred)).sum()
  test_loss /= len(test_loader.dataset)
  test_losses.append(test_loss)
  test_accuracies.append(100. * correct / len(test_loader.dataset))
  print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

In [7]:
# # tNet = torch.load('tNet.pt')
# fNet = FastUpdateNet(teacherNet=tNet)
# test(fNet)
# fNet = TeacherNet()
# optimizer = optim.Adam(fNet.parameters(), lr=learning_rate)
# train_losses = []
# train_counter = []
# test_losses = []
# test_accuracies = []

# test(fNet)
# for epoch in range(1, n_epochs + 1):
#   train_teacherNet(epoch, fNet)
#   test(fNet)

baseline using no M with Adam and lr=0.0001


Test set: Avg. loss: 2.3070, Accuracy: 958/10000 (10%)


Test set: Avg. loss: 0.5891, Accuracy: 8403/10000 (84%)


Test set: Avg. loss: 0.3535, Accuracy: 8988/10000 (90%)


Test set: Avg. loss: 0.2795, Accuracy: 9184/10000 (92%)


Test set: Avg. loss: 0.2455, Accuracy: 9275/10000 (93%)


Test set: Avg. loss: 0.2153, Accuracy: 9369/10000 (94%)


Test set: Avg. loss: 0.1997, Accuracy: 9394/10000 (94%)


Test set: Avg. loss: 0.1828, Accuracy: 9463/10000 (95%)


Test set: Avg. loss: 0.1655, Accuracy: 9501/10000 (95%)


Test set: Avg. loss: 0.1535, Accuracy: 9538/10000 (95%)


Test set: Avg. loss: 0.1458, Accuracy: 9562/10000 (96%)


Test set: Avg. loss: 0.1372, Accuracy: 9581/10000 (96%)


Test set: Avg. loss: 0.1307, Accuracy: 9609/10000 (96%)


Test set: Avg. loss: 0.1247, Accuracy: 9613/10000 (96%)


Test set: Avg. loss: 0.1175, Accuracy: 9637/10000 (96%)


Test set: Avg. loss: 0.1117, Accuracy: 9658/10000 (97%)


Test set: Avg. loss: 0.1171, Accuracy: 9647/10000 (96%)


Test set: Avg. loss: 0.1044, Accuracy: 9683/10000 (97%)


Test set: Avg. loss: 0.1066, Accuracy: 9668/10000 (97%)


Test set: Avg. loss: 0.1003, Accuracy: 9692/10000 (97%)


Test set: Avg. loss: 0.0949, Accuracy: 9707/10000 (97%)

In [8]:
fNet = FastUpdateNet()
# # optimizer = optim.SGD(fNet.get_parameters(), lr=learning_rate, momentum=momentum)
optimizer = optim.Adam(fNet.get_parameters(), lr=learning_rate)

with inv
error tensor(0.0003, grad_fn=<SumBackward0>)
0.018805503845214844

with pinv
error tensor(0.00004, grad_fn=<SumBackward0>)
0.02195906639099121

0.01661086082458496

In [9]:
train_losses = []
train_counter = []
test_losses = []
test_accuracies = []

test(fNet)
for epoch in range(1, n_epochs + 1):
  train_fastNet(epoch, fNet)
  test(fNet)

/home/kyle/projects/cs674/magic-m/model/network.py:157: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(o_4)
/home/kyle/pythonenvs/venv38/lib/python3.10/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Avg. loss: 2.3087, Accuracy: 959/10000 (10%)


Test set: Avg. loss: 1.7311, Accuracy: 5568/10000 (56%)


Test set: Avg. loss: 0.7874, Accuracy: 7854/10000 (79%)


Test set: Avg. loss: 0.5601, Accuracy: 8435/10000 (84%)


Test set: Avg. loss: 0.4566, Accuracy: 8730/10000 (87%)


Test set: Avg. loss: 0.4164, Accuracy: 8822/10000 (88%)


Test set: Avg. loss: 0.3799, Accuracy: 8918/10000 (89%)


Test set: Avg. loss: 0.3400, Accuracy: 9023/10000 (90%)


Test set: Avg. loss: 0.3179, Accuracy: 9080/10000 (91%)


Test set: Avg. loss: 0.3143, Accuracy: 9088/10000 (91%)


Test set: Avg. loss: 0.2928, Accuracy: 9146/10000 (91%)


Test set: Avg. loss: 0.2879, Accuracy: 9150/10000 (92%)


Test set: Avg. loss: 0.2734, Accuracy: 9203/10000 (92%)


Test set: Avg. loss: 0.2622, Accuracy: 9228/10000 (92%)


Test set: Avg. loss: 0.2558, Accuracy: 9254/10000 (93%)


Test set: Avg. loss: 0.2488, Accuracy: 9263/10000 (93%)


Test set: Avg. loss: 0.2388, Accuracy: 9291/10000 (93%)


Test set: Avg.

using naive M calc (pseudoinverse) with Adam with lr=0.0001

Test set: Avg. loss: 2.3064, Accuracy: 958/10000 (10%)


Test set: Avg. loss: 0.9090, Accuracy: 7807/10000 (78%)


Test set: Avg. loss: 0.4521, Accuracy: 8727/10000 (87%)


Test set: Avg. loss: 0.3442, Accuracy: 8977/10000 (90%)


Test set: Avg. loss: 0.2926, Accuracy: 9111/10000 (91%)


Test set: Avg. loss: 0.2553, Accuracy: 9217/10000 (92%)


Test set: Avg. loss: 0.2276, Accuracy: 9312/10000 (93%)


Test set: Avg. loss: 0.2112, Accuracy: 9357/10000 (94%)


Test set: Avg. loss: 0.1993, Accuracy: 9399/10000 (94%)


Test set: Avg. loss: 0.1912, Accuracy: 9412/10000 (94%)


Test set: Avg. loss: 0.1830, Accuracy: 9438/10000 (94%)


Test set: Avg. loss: 0.1754, Accuracy: 9468/10000 (95%)


Test set: Avg. loss: 0.1695, Accuracy: 9490/10000 (95%)


Test set: Avg. loss: 0.1624, Accuracy: 9515/10000 (95%)


Test set: Avg. loss: 0.1584, Accuracy: 9528/10000 (95%)


Test set: Avg. loss: 0.1499, Accuracy: 9555/10000 (96%)


Test set: Avg. loss: 0.1460, Accuracy: 9557/10000 (96%)


Test set: Avg. loss: 0.1425, Accuracy: 9560/10000 (96%)


Test set: Avg. loss: 0.1368, Accuracy: 9584/10000 (96%)


Test set: Avg. loss: 0.1346, Accuracy: 9592/10000 (96%)


Test set: Avg. loss: 0.1293, Accuracy: 9603/10000 (96%)

Using split M calc with Adam and lr=0.0001 and lstsq instead of solve

Test set: Avg. loss: 2.3068, Accuracy: 1019/10000 (10%)


Test set: Avg. loss: 1.4663, Accuracy: 6400/10000 (64%)


Test set: Avg. loss: 0.5511, Accuracy: 8553/10000 (86%)


Test set: Avg. loss: 0.3758, Accuracy: 8914/10000 (89%)


Test set: Avg. loss: 0.3146, Accuracy: 9099/10000 (91%)


Test set: Avg. loss: 0.2726, Accuracy: 9209/10000 (92%)


Test set: Avg. loss: 0.2409, Accuracy: 9293/10000 (93%)


Test set: Avg. loss: 0.2143, Accuracy: 9375/10000 (94%)


Test set: Avg. loss: 0.1941, Accuracy: 9438/10000 (94%)


Test set: Avg. loss: 0.1769, Accuracy: 9474/10000 (95%)


Test set: Avg. loss: 0.1626, Accuracy: 9512/10000 (95%)


Test set: Avg. loss: 0.1504, Accuracy: 9539/10000 (95%)


Test set: Avg. loss: 0.1427, Accuracy: 9578/10000 (96%)


Test set: Avg. loss: 0.1326, Accuracy: 9596/10000 (96%)


Test set: Avg. loss: 0.1267, Accuracy: 9632/10000 (96%)


Test set: Avg. loss: 0.1196, Accuracy: 9639/10000 (96%)


Test set: Avg. loss: 0.1159, Accuracy: 9662/10000 (97%)


Test set: Avg. loss: 0.1093, Accuracy: 9676/10000 (97%)


Test set: Avg. loss: 0.1040, Accuracy: 9685/10000 (97%)


Test set: Avg. loss: 0.1023, Accuracy: 9682/10000 (97%)


Test set: Avg. loss: 0.0981, Accuracy: 9697/10000 (97%)

using split M calc (94%) with Adam and lr=0.0001 and small perturbation to move diagonals away from 0

Test set: Avg. loss: 2.3053, Accuracy: 1341/10000 (13%)


Test set: Avg. loss: 1.5690, Accuracy: 6727/10000 (67%)


Test set: Avg. loss: 0.6742, Accuracy: 8214/10000 (82%)


Test set: Avg. loss: 0.4693, Accuracy: 8699/10000 (87%)


Test set: Avg. loss: 0.3940, Accuracy: 8880/10000 (89%)


Test set: Avg. loss: 0.3485, Accuracy: 9012/10000 (90%)


Test set: Avg. loss: 0.3197, Accuracy: 9080/10000 (91%)


Test set: Avg. loss: 0.3069, Accuracy: 9103/10000 (91%)


Test set: Avg. loss: 0.2885, Accuracy: 9164/10000 (92%)


Test set: Avg. loss: 0.2753, Accuracy: 9215/10000 (92%)


Test set: Avg. loss: 0.2627, Accuracy: 9233/10000 (92%)


Test set: Avg. loss: 0.2469, Accuracy: 9310/10000 (93%)


Test set: Avg. loss: 0.2411, Accuracy: 9316/10000 (93%)


Test set: Avg. loss: 0.2346, Accuracy: 9320/10000 (93%)


Test set: Avg. loss: 0.2316, Accuracy: 9322/10000 (93%)


Test set: Avg. loss: 0.2276, Accuracy: 9313/10000 (93%)


Test set: Avg. loss: 0.2195, Accuracy: 9346/10000 (93%)


Test set: Avg. loss: 0.2138, Accuracy: 9375/10000 (94%)


Test set: Avg. loss: 0.2090, Accuracy: 9372/10000 (94%)


Test set: Avg. loss: 0.2070, Accuracy: 9397/10000 (94%)


Test set: Avg. loss: 0.2008, Accuracy: 9423/10000 (94%)

In [10]:
# without our method batchsize 512 
Test set: Avg. loss: 2.3055, Accuracy: 984/10000 (10%)


Test set: Avg. loss: 2.2689, Accuracy: 1683/10000 (17%)


Test set: Avg. loss: 2.0823, Accuracy: 3535/10000 (35%)


Test set: Avg. loss: 1.1515, Accuracy: 6768/10000 (68%)


Test set: Avg. loss: 0.6305, Accuracy: 8208/10000 (82%)


Test set: Avg. loss: 0.4735, Accuracy: 8594/10000 (86%)


Test set: Avg. loss: 0.4033, Accuracy: 8820/10000 (88%)


Test set: Avg. loss: 0.3620, Accuracy: 8917/10000 (89%)


Test set: Avg. loss: 0.3314, Accuracy: 9027/10000 (90%)


Test set: Avg. loss: 0.3024, Accuracy: 9092/10000 (91%)


Test set: Avg. loss: 0.2778, Accuracy: 9175/10000 (92%)


Test set: Avg. loss: 0.2556, Accuracy: 9236/10000 (92%)


Test set: Avg. loss: 0.2465, Accuracy: 9245/10000 (92%)


Test set: Avg. loss: 0.2244, Accuracy: 9322/10000 (93%)


Test set: Avg. loss: 0.2122, Accuracy: 9360/10000 (94%)


Test set: Avg. loss: 0.2006, Accuracy: 9403/10000 (94%)


Test set: Avg. loss: 0.1868, Accuracy: 9436/10000 (94%)


Test set: Avg. loss: 0.1802, Accuracy: 9443/10000 (94%)


Test set: Avg. loss: 0.1739, Accuracy: 9460/10000 (95%)


Test set: Avg. loss: 0.1652, Accuracy: 9514/10000 (95%)


Test set: Avg. loss: 0.1587, Accuracy: 9521/10000 (95%)


SyntaxError: invalid syntax (3724229338.py, line 2)

In [ ]:
a = torch.nn.Linear(2 , 3)
# print(a.weight.shape)
x = torch.rand((2, 2))
y = a(x)
external_grad = torch.rand((2, 3))
y.backward(gradient = external_grad)

# print(y.shape)
print(a.weight.grad)
print(torch.matmul( a.weight, external_grad))
# print(external_grad * a.weight)

print(torch.matmul(torch.transpose(external_grad, 0, 1), x))
print(external_grad)

tensor([[0.8751, 0.7492],
        [0.5353, 0.2133],
        [0.8195, 0.2682]])
tensor([[0.5097, 0.3156, 0.4840],
        [0.1640, 0.0265, 0.0229],
        [0.3872, 0.1735, 0.2506]], grad_fn=<MmBackward0>)
tensor([[0.8751, 0.7492],
        [0.5353, 0.2133],
        [0.8195, 0.2682]])
tensor([[7.4310e-01, 5.4924e-01, 8.6340e-01],
        [5.6135e-01, 4.6440e-02, 2.4873e-04]])


In [ ]:
tNet = torch.load('tNet.pt')
fNet = FastUpdateNet(teacherNet=tNet)
fNet.fc1 = torch.nn.Linear(784, 392)
optimizer = optim.SGD(fNet.fc1.parameters(), lr=learning_rate, momentum=momentum)
test(fNet)
for epoch in range(1, 4):
  train_teacherNet(epoch, fNet)
  test(fNet)

FileNotFoundError: [Errno 2] No such file or directory: 'tNet.pt'

In [ ]:
import torch.nn as nn

In [ ]:
fNet = FastUpdateNet()
fNet.mNet = torch.nn.Sequential(nn.Linear(392, 196), nn.ReLU(), nn.Linear(196, 98), nn.ReLU(), nn.Linear(98, 49), nn.ReLU())
optimizer = optim.SGD(fNet.parameters(), lr=learning_rate, momentum=momentum)

In [ ]:
train_losses = []
train_counter = []
test_losses = []
test_accuracies = []

test(fNet)
for epoch in range(1, n_epochs + 1):
  train_teacherNet(epoch, fNet)
  test(fNet)

/home/hao/Documents/magic-m/model/network.py:131: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(o_4)
/home/hao/.local/lib/python3.8/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Avg. loss: 2.3096, Accuracy: 891/10000 (9%)


Test set: Avg. loss: 0.3962, Accuracy: 8793/10000 (88%)


Test set: Avg. loss: 0.2001, Accuracy: 9407/10000 (94%)


Test set: Avg. loss: 0.1858, Accuracy: 9440/10000 (94%)


Test set: Avg. loss: 0.1283, Accuracy: 9602/10000 (96%)

